<a href="https://colab.research.google.com/github/syedsaqib122/LangChain-Hello-World/blob/main/LanngChain_RAG(Retrival_Augmented_Generation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LangChainRag**

In [3]:
%pip install -Uq Langchain-pinecone Langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 1.3 MB/s eta 0:00:00


In [5]:
import os
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
pc = Pinecone(api_key=PINECONE_API_KEY)



In [6]:
import pinecone

index_name = "langchain-rag-project"

# Check if the index exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=pinecone.ServerlessSpec(cloud="aws", region="us-east-1"),
    )
else:
    print(f"Index '{index_name}' already exists.")

# Connect to the existing index
index = pc.Index(index_name)


In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

os.environ['GOOGLE_API_KEY']= userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[0:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [12]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore( index=index, embedding=embeddings,)

In [13]:
#Dummy Documents
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [14]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['16c60848-83d6-4d47-9c8f-1dddefc829f0',
 '5a1eca04-8537-4042-aefa-8429be0613a8',
 '628ee9ff-1e0f-4637-a109-41bc39743a97',
 'a638359b-32e0-43fa-99b2-cf7267d2dec1',
 '7f6b35ac-4069-4b5b-b070-18a3c611d1ba',
 'f3e0149b-3c7e-43c2-901d-c355d7ab55e6',
 '01ec7d27-2807-4cdc-82df-88194d8df27c',
 'f40be8bd-9770-4d80-9ec2-94befae5f9dc',
 'cfe3b635-bf2a-4fc0-9d85-25635ba182a4',
 '20e01c04-5f33-40e9-922a-e89b37660898']

In [15]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
     k=2.,
    filter={"source": "tweet"},

)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [16]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=5, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.533720] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]
* [SIM=0.474582] Robbers broke into the city bank and stole $1 million in cash. [{'source': 'news'}]


In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [18]:
def question_answering(question: str):
  results = vector_store.similarity_search(question, k=2)

  final_answer = llm.invoke(f"ANSWER THIS USER QUERRY:{question} USING THE CONTEXT:{results}")
  return final_answer

In [19]:
ask_a_question = input("Ask a question: ")
answer = question_answering(ask_a_question)
print(answer)


Ask a question: what is the capital of pakistan
content='This question cannot be answered from the given source.  The provided text does not contain any information about the capital of Pakistan.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-a66edcb7-34aa-45be-a728-50c2f2957c00-0' usage_metadata={'input_tokens': 147, 'output_tokens': 26, 'total_tokens': 173, 'input_token_details': {'cache_read': 0}}
